In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Cấu hình hiển thị
pd.set_option('display.max_columns', None)
print("import thành công")

import thành công


# 2. TẢI DỮ LIỆU

In [2]:
file_path = '../data/raw/superstore.csv' 
try:
    # Nếu chạy lỗi file path
    df = pd.read_csv(file_path)
    print(f"Đã tải dữ liệu thành công. Kích thước: {df.shape}")
except FileNotFoundError:
    print("Không tìm thấy file. Vui lòng kiểm tra lại đường dẫn.")
    # test nhanh
    df = pd.read_csv('superstore.csv')

Đã tải dữ liệu thành công. Kích thước: (51290, 27)


# 3. Tổng Quan

In [3]:
print("\n--- 3.1. THÔNG TIN CẤU TRÚC (INFO) ---")
print(df.info())

print("\n--- 3.2. MẪU DỮ LIỆU (HEAD) ---")
display(df.head(3))


--- 3.1. THÔNG TIN CẤU TRÚC (INFO) ---
<class 'pandas.DataFrame'>
RangeIndex: 51290 entries, 0 to 51289
Data columns (total 27 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Category        51290 non-null  str    
 1   City            51290 non-null  str    
 2   Country         51290 non-null  str    
 3   Customer.ID     51290 non-null  str    
 4   Customer.Name   51290 non-null  str    
 5   Discount        51290 non-null  float64
 6   Market          51290 non-null  str    
 7   记录数             51290 non-null  int64  
 8   Order.Date      51290 non-null  str    
 9   Order.ID        51290 non-null  str    
 10  Order.Priority  51290 non-null  str    
 11  Product.ID      51290 non-null  str    
 12  Product.Name    51290 non-null  str    
 13  Profit          51290 non-null  float64
 14  Quantity        51290 non-null  int64  
 15  Region          51290 non-null  str    
 16  Row.ID          51290 non-null  int64  
 17  Sa

,Category,City,Country,Customer.ID,Customer.Name,Discount,Market,记录数,Order.Date,Order.ID,Order.Priority,Product.ID,Product.Name,Profit,Quantity,Region,Row.ID,Sales,Segment,Ship.Date,Ship.Mode,Shipping.Cost,State,Sub.Category,Year,Market2,weeknum
0,Office Supplies,Los Angeles,United States,LS-172304,Lycoris Saunders,0.0,US,1,2011-01-07 00:00:00.000,CA-2011-130813,High,OFF-PA-10002005,Xerox 225,9.3312,3,West,36624,19,Consumer,2011-01-09 00:00:00.000,Second Class,4.37,California,Paper,2011,North America,2
1,Office Supplies,Los Angeles,United States,MV-174854,Mark Van Huff,0.0,US,1,2011-01-21 00:00:00.000,CA-2011-148614,Medium,OFF-PA-10002893,"Wirebound Service Call Books, 5 1/2"" x 4""",9.2928,2,West,37033,19,Consumer,2011-01-26 00:00:00.000,Standard Class,0.94,California,Paper,2011,North America,4
2,Office Supplies,Los Angeles,United States,CS-121304,Chad Sievert,0.0,US,1,2011-08-05 00:00:00.000,CA-2011-118962,Medium,OFF-PA-10000659,"Adams Phone Message Book, Professional, 400 Me...",9.8418,3,West,31468,21,Consumer,2011-08-09 00:00:00.000,Standard Class,1.81,California,Paper,2011,North America,32


# 4. Kiểm tra chất lượng data 

In [4]:
print("\n--- 4.1. GIÁ TRỊ THIẾU (MISSING VALUES) ---")
missing = df.isnull().sum()
missing = missing[missing > 0]
if missing.empty:
    print("Không có giá trị thiếu (Null) nào.")
else:
    print("Các cột sau có dữ liệu thiếu:")
    print(missing)
    print("-> Cần xử lý trong bước Cleaning (Điền mặc định hoặc Xóa).")


--- 4.1. GIÁ TRỊ THIẾU (MISSING VALUES) ---
Không có giá trị thiếu (Null) nào.


In [5]:
# 4.2. Kiểm tra Trùng lặp (Duplicates)
print("\n--- 4.2. DỮ LIỆU TRÙNG LẶP (DUPLICATES) ---")
# Check 1: Trùng lặp hoàn toàn (Full Row Duplicates)
dup_rows = df.duplicated().sum()
print(f"- Số dòng trùng lặp hoàn toàn: {dup_rows}")


--- 4.2. DỮ LIỆU TRÙNG LẶP (DUPLICATES) ---
- Số dòng trùng lặp hoàn toàn: 0


In [6]:
# Check 2: Trùng lặp Business Key (Order ID + Product ID)
# Một Order không nên có 2 dòng cho cùng 1 Product ID (trừ khi tách shipping)
business_dups = df.duplicated(subset=['Order.ID', 'Product.ID']).sum()
print(f"- Số dòng trùng lặp Business Key (Order ID + Product ID): {business_dups}")

if business_dups > 0:
    print("-> Có khả năng lỗi nhập liệu. Cần gộp.")

- Số dòng trùng lặp Business Key (Order ID + Product ID): 38
-> Có khả năng lỗi nhập liệu. Cần gộp.


In [7]:
# 4.3. Kiểm tra Logic Thời gian (Time Consistency)
print("\n--- 4.3. LOGIC THỜI GIAN (TIME LOGIC) ---")
# Chuyển đổi tạm thời sang datetime để check
df['Order.Date.Temp'] = pd.to_datetime(df['Order.Date'])
df['Ship.Date.Temp'] = pd.to_datetime(df['Ship.Date'])

# Logic: Ngày giao hàng < Ngày đặt hàng?
invalid_dates = df[df['Ship.Date.Temp'] < df['Order.Date.Temp']]
num_invalid_dates = len(invalid_dates)
print(f"- Số đơn hàng có Ngày Ship < Ngày Đặt: {num_invalid_dates}")
if num_invalid_dates > 0:
    print("-> ⚠️ Dữ liệu thời gian bị sai logic. Cần xóa hoặc sửa.")

# Xóa cột tạm
df.drop(columns=['Order.Date.Temp', 'Ship.Date.Temp'], inplace=True)


--- 4.3. LOGIC THỜI GIAN (TIME LOGIC) ---
- Số đơn hàng có Ngày Ship < Ngày Đặt: 0
